# Lesson 6. Advanced Parameter Estimation

In [2]:
import pytherm #Required

import numpy as np #Recommended
from matplotlib import pyplot as plt #Recommended - used for plotting

In [3]:
#Here we initialize the class
exp_data = pytherm.Experimental_Data()

#By using the "Add" class function, we may add experimental data to the class. First we add vapor pressure data
filename_psat = 'experimental_data/TEG_vapor_pressure.csv'
datatype_psat = 'PSat'
identifier_psat = 'teg_psat'
exp_data.Add(filename_psat, datatype_psat, identifier_psat)

#Similarly, we add liquid density data.
filename_rho = 'experimental_data/TEG_liq_density.csv'
datatype_rho = 'rho'
identifier_rho = 'teg_rho'
exp_data.Add(filename_rho, datatype_rho, identifier_rho)



In [4]:
#It is common to limit the pure component data to a specific temperature range, typically expressed as an
#an interval of reduced temperatures. If we seek to remove data outside these region, we impose the interval by 
#using the ReducedTemperature function

Tr_min = 0.4 #Reduced temperature min
Tr_max = 0.9 #Reduced temperature max
Tc = 769.5 #Critical temprature [K]
exp_data.ReducedTemperature(Tr_min, Tr_max, Tc)

print("After adjusting for reduced temperature")
exp_data.Show_list()

After adjusting for reduced temperature
Data Type        Dim (r x c)     Name           
-------------------------------------------------------------
PSat             38x2            teg_psat       
rho              50x2            teg_rho        




In [5]:
crit_props = {
    "Tc" : 769.5,
    "Pc" : 33.2,
    "Om" : 0.05 
}

original_params = {
    "b0" : 128.93,
    "Gamma" : 3622.5,
    "c1" : 0.9100,
    "AssocEng" :  1420.0,
    "AssocVol" : 20.0
}

AssocSch = 24 #Corresponds to 6D associaton scheme

In [12]:
#First, the thermodynamic calculations must be set up as per usua
Thermo = pytherm.Model()

Thermo.NoPureComp(1)
Thermo.CritProps(1,crit_props["Tc"], crit_props["Pc"], crit_props["Om"])
Thermo.CPAParams(1,original_params["b0"],original_params["Gamma"],original_params["c1"])
Thermo.AssocParams(1,AssocSch,original_params["AssocVol"],original_params["AssocEng"])

### Selecting an optimization algorithm

In [7]:
#From here we create a Optimizer object
optimizer = pytherm.Optimizer()

#The optimizer is now given a model to use, the Model object
optimizer.Add_Model(Thermo)

#Now we add the experimental data object to the Optimizer object
optimizer.Add_Experimental_Data(exp_data)

#Now we set the optimization algorithm
algorithm = "trf" # lm - levenberg marquardt, trf - trust region reflective, dogbox - dogleg algorithm
optimizer.Set_Optimization_Algorithm(algorithm)

#Now the optimizer have enough information to perform parameterization. Keep in mind that Optimizer is solely for
#CPA model, a
opti_params = optimizer.Calculation()

In [7]:
print(opti_params)

{'b0': 129.24402327699477, 'Gamma': 3640.745156582866, 'c1': 1.2114765046901492, 'AssocVol': 16.707300848909743, 'AssocEng': 1281.8909212078538}


### Setting upper and lower bounds for decision variables

In [19]:
#From here we create a Optimizer object
optimizer = pytherm.Optimizer()

#The optimizer is now given a model to use, the Model object
optimizer.Add_Model(Thermo)

#Now we add the experimental data object to the Optimizer object
optimizer.Add_Experimental_Data(exp_data)

#Now we set the optimization algorithm (remember, levenberg-marquardt does NOT work with bounds)
algorithm = "trf" # lm - levenberg marquardt, trf - trust region reflective, dogbox - dogleg algorithm
optimizer.Set_Optimization_Algorithm(algorithm)

bounds = {
    "b0" : [125,135],
    "Gamma" : [3000,4000],
    "c1" : [0.5,1.0],
    "AssocVol" : [10,30],
    "AssocEng" : [1000, 2000]
}
optimizer.Add_Bounds(bounds)

#Now the optimizer have enough information to perform parameterization. Keep in mind that Optimizer is solely for
#CPA model, a
opti_params = optimizer.Calculation()

In [20]:
print(opti_params)

{'b0': 128.5067444497707, 'Gamma': 3166.894845438052, 'c1': 0.9998783966721797, 'AssocVol': 14.766484773366008, 'AssocEng': 1873.89259257215}


### How to use multiple initial conditions for finding global minimum

In [9]:
#From here we create a Optimizer object
optimizer = pytherm.Optimizer()

#The optimizer is now given a model to use, the Model object
optimizer.Add_Model(Thermo)

#Now we add the experimental data object to the Optimizer object
optimizer.Add_Experimental_Data(exp_data)

#Now we set the optimization algorithm
algorithm = "trf" # lm - levenberg marquardt, trf - trust region reflective, dogbox - dogleg algorithm
optimizer.Set_Optimization_Algorithm(algorithm)

bounds = {
    "b0" : [125,135],
    "Gamma" : [3000,4000],
    "c1" : [0.5,1.5],
    "AssocVol" : [10,30],
    "AssocEng" : [1000, 2000]
}
iterations = 20
optimizer.Setup_Multistart(bounds, iterations)

opti_params = optimizer.Calculation(MultiStart = True) 

Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\s164067\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-306db357c64a>", line 24, in <module>
    opti_params = optimizer.Calculation(MultiStart = True)
  File "C:\Users\s164067\Desktop\CERE_PYTHON_PROJECT\pytherm.py", line 1498, in Calculation
    out = least_squares(self.__Residual, variables, args = (), method = self.algorithm, bounds = self.bounds_var)
  File "C:\Users\s164067\anaconda3\lib\site-packages\scipy\optimize\_lsq\least_squares.py", line 915, in least_squares
    result = trf(fun_wrapped, jac_wrapped, x0, f0, J0, lb, ub, ftol, xtol,
  File "C:\Users\s164067\anaconda3\lib\site-packages\scipy\optimize\_lsq\trf.py", line 119, in trf
    return trf_no_bounds(
  File "C:\Users\s164067\anaconda3\lib\site-packages\scipy\optimize\_lsq\trf.py", line 536, in trf_no_bounds
    J = jac(x, f)
  File "C:\Users\s164067\an

TypeError: object of type 'NoneType' has no len()

In [ ]:
print(opti_params)

In [ ]:
n = 100


exp_psat = exp_data.Retrieve_data('teg_psat')
exp_rho = exp_data.Retrieve_data('teg_rho')

expT_psat = exp_psat[:,0]
expP = exp_psat[:,1]

expT_rho = exp_rho[:,0]
expRho = exp_rho[:,1]

T_model_psat = np.linspace(min(expT_psat),max(expT_psat),n)
T_model_rho = np.linspace(min(expT_rho),max(expT_rho),n)
P_model_imp = np.zeros(n) #Memory allocation - Good practice, more efficient
rho_model_imp = np.zeros(n) #Memory allocation - Good practice, more efficient


Thermo_improved = pytherm.Model()
Thermo_improved.NoPureComp(1)
Thermo_improved.CritProps(1,crit_props["Tc"], crit_props["Pc"], crit_props["Om"])
Thermo_improved.CPAParams(1,opti_params["b0"],opti_params["Gamma"],opti_params["c1"])
Thermo_improved.AssocParams(1,AssocSch,opti_params["AssocVol"],opti_params["AssocEng"])

Thermo_improved.Setup_Thermo()

for i in range(0,n):
    P_model_imp[i], b, c = Thermo_improved.PBubble(T_model_psat[i],[1.0])
    rho_model_imp[i] = Thermo.LiqRho(T_model_rho[i],[1.0])

Thermo_improved.Finishup_Thermo()
#-----------------------------------------------------------------------------------

#plt.plot(T_model_psat,P_model,color="black")
plt.plot(T_model_psat,P_model_imp,color="red")
plt.plot(expT_psat,expP,'o',color="blue",markersize=  6)
plt.xlabel("Temperature [K]")
plt.ylabel("Vapor Pressure [bar]")
plt.yscale('log')

plt.legend(["Before optimization","After optimization","Experimental data"])
plt.grid()
plt.show()

#plt.plot(T_model_rho,rho_model,color="black")
plt.plot(T_model_rho,rho_model_imp,color="red")
plt.plot(expT_rho,expRho,'o',color="blue",markersize=  6)
plt.xlabel("Temperature [K]")
plt.ylabel("Liquid Density [mol/L]")
plt.legend(["Before optimization","After optimization","Experimental data"])
plt.grid()
plt.show()